<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas - Emailbuilder demo
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Emailbuilder_demo.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+-+Emailbuilder+demo:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook provides a demonstration of the Naas Emailbuilder, a tool for creating and managing email campaigns.

## Input

### Import libraries

In [1]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime
from naas_drivers import naasauth

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.


### Setup variables
- `linkedin_url`: This variable is used for storing the URL of the LinkedIn profile to be updated. It is retrieved from an environment variable called "LINKEDIN_PROFILE_URL" using the os.environ.get() method. If the environment variable is not set, the naas.secret.get() method is used to retrieve the URL from a secret store.
- `output_dir`: This variable is used for storing the path to the directory where the output files of the LinkedIn profile update process will be saved. In this case, the path is set to "/home/ftp/content-engine-linkedin-profile/outputs/linkedin/profile".
- `subdir`: This variable is used for storing the name of a subdirectory where the output files will be saved. In this case, the subdirectory is set to "12_mr_month".
- `graphs`: This variable is used for storing a list of graph types that will be updated for the LinkedIn profile. The available graph types are "published", "views", "likes", "comments", and "shares".
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email. In this case, the list contains the email address of the script user, which is retrieved using the naasauth.connect().user.me().get("username") method.
- `subject`: This variable is used for storing the subject of the notification email. In this case, the subject is "🖼️ Content Engine: LinkedIn profile update".
- `email_content`: This variable is used for storing the content of the notification email. In this case, it is an empty dictionary, which can be populated with additional content later.

In [2]:
# Inputs
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1GDVJxBajYx7VR48yKGnSvMsEjzXvqI94hlmPXI5qEp8/edit#gid=0" or naas.secret.get("MY_AI_AVATAR_SPREADSHEET")
sheet_name = "CONTENT"
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
image_input = os.path.join(input_dir, f"views.png")
image_url = naas.asset.find(image_input)
plugin_url = naas.asset.find(os.path.join(naas_data_product.OUTPUTS_PATH, "plugins", "📲_content_assistant.json"))
limit = 10

# Outputs
email_to = "jeremy@naas.ai"
email_subject = f"📲 Content Assistant - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
email_content = {}

## Model

### Get data from Google Sheets spreadsheet

In [3]:
df = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)[:limit]
print("Rows:", len(df))
df.head(1)

Rows: 10


,ENTITY,SCENARIO,SOURCE,PUBLISHED_DATE,DATE,TIME,TITLE,CONTENT,CONTENT_LENGTH,KEYWORDS,VIEWS,LIKES,COMMENTS,SHARES,ENGAGEMENT_SCORE,CONTENT_URL
0,Jérémy Ravenel,2023-11-24,LinkedIn,2023-11-23 21:02:47+0100,2023-11-23,21:02:47,Happy Thanksgiving everyone. 🍁🍂🥧,Happy Thanksgiving everyone. 🍁🍂🥧\nMay this day...,846,None,961,12,2,0,0.0146,https://www.linkedin.com/feed/update/urn:li:ac...


### Prep values for email

In [4]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f}"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

views = format_number(df.VIEWS.sum())
likes = format_number(df.LIKES.sum())
comments = format_number(df.COMMENTS.sum())
shares = format_number(df.SHARES.sum())

# preview of what will be send by email:
print(f"On the past {limit} pieces of content:")
print(
    "\n\t👀 Impressions\t",
    views,
    "\n\t👍 Likes\t",
    likes,
    "\n\t💬 Comments\t",
    comments,
    "\n\t⏩ Shares\t",
    shares,
)

On the past 10 pieces of content:

	👀 Impressions	 24 820 
	👍 Likes	 248 
	💬 Comments	 100 
	⏩ Shares	 11


### Create Email brief

In [ ]:
def email_brief(
    views,
    likes,
    comments,
    shares,
    plugin_url,
    image_url,
    limit,
):
    content = {
        'title': "📲 Content Assistant - Email Update",
        'heading': (f"Date: {date.today().isoformat()}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your content engine email as of {date.today().isoformat()}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url),
        "text_1": emailbuilder.text(
            f"On the past {limit} pieces of content, you reached a total of:"
        ),
        "list_1": emailbuilder.list(
            [
                f"Views : <b>{views}</b>",
                f"Likes : <b>{likes}</b>",
                f"Comments : <b>{comments}</b>",
                f"Shares : <b>{shares}</b>",
            ]
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting With Assistant",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    views,
    likes,
    comments,
    shares,
    plugin_url,
    image_url,
    limit,
)

## Output

### Send the email

In [ ]:
naas.notification.send(
    email_to=email_to,
    subject=email_subject,
    html=email_content,
)